In [1]:
from exp.Preprocessing import *
from exp.dl_utils import *
from exp.control_flow_prediction import *

In [2]:
%%time
train_df, test_df = preprocess_and_train_test_split(URLs.BPIC_2012,cache=True,override=True)
train_df.head(5)

Successfully created the directory ./.cache/datasets/BPI_Challenge_2012
CPU times: user 39 s, sys: 656 ms, total: 39.6 s
Wall time: 39.5 s


,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp,REG_DATE,AMOUNT_REQ,activity:name,activity,resource,relative timestamp,next relative timestamp,duration to end,next activity,next resource,last resource,outcome,activity suffix,resource suffix
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-10-01 00:38:44.546000+02:00,20000,A_SUBMITTED_COMPLETE,11,54,0.334,53.360,1072732.480,8,54,8,34,"[8, 9, 26, 27, 2, 16, 7, 14, 18, 32, 25, 33, 3...","[54, 54, 54, 70, 15, 15, 15, 15, 15, 70, 70, 7..."
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-10-01 00:38:44.546000+02:00,20000,A_PARTLYSUBMITTED_COMPLETE,8,54,53.026,54.329,1072732.146,9,54,8,34,"[9, 26, 27, 2, 16, 7, 14, 18, 32, 25, 33, 31, ...","[54, 54, 70, 15, 15, 15, 15, 15, 70, 70, 70, 7..."
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-10-01 00:38:44.546000+02:00,20000,A_PREACCEPTED_COMPLETE,9,54,0.969,39481.891,1072679.120,26,54,8,34,"[26, 27, 2, 16, 7, 14, 18, 32, 25, 33, 31, 33,...","[54, 70, 15, 15, 15, 15, 15, 70, 70, 70, 70, 2..."
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00,2011-10-01 00:38:44.546000+02:00,20000,W_Completeren aanvraag_SCHEDULE,26,54,39427.562,39838.762,1072678.151,27,70,8,34,"[27, 2, 16, 7, 14, 18, 32, 25, 33, 31, 33, 31,...","[70, 15, 15, 15, 15, 15, 70, 70, 70, 70, 24, 2..."
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00,2011-10-01 00:38:44.546000+02:00,20000,W_Completeren aanvraag_START,27,70,356.871,39984.697,1033250.589,2,15,8,34,"[2, 16, 7, 14, 18, 32, 25, 33, 31, 33, 31, 33,...","[15, 15, 15, 15, 15, 70, 70, 70, 70, 24, 24, 4..."


In [362]:
def create_traces(event_df, trace_id='trace_id'):
    ll=[]
    trace_ids=[]
    cols=list(event_df)
    cols.remove(trace_id)
    for n, g in event_df.groupby(trace_id): #progress_bar()
        l=[]
        for c in cols:
            l.append(list(g[c]))
        ll.append(l)
        trace_ids.append(n)  
    df=pd.DataFrame(ll,columns=cols)
    df["trace_id"] = trace_ids
    return df

def drop_short_cases(traces, col, index=2):
    return traces[traces[col].map(len) > (index+1)].reset_index(drop=True)

def build_windows(traces, ws, col, index=2):
    val = traces[col].values
    X = []
    y = []

    for i in range(len(traces)):
        for j in range(len(val[i])):
            if j+ws >= len(val[i]) or j < ws-1: #j+ws+1, ws-1
                continue
            else:
                windows = []
                windows.append(traces["trace_id"][i]) # trace_id
                windows.append(val[i][j:ws+j])        # windows
                y.append(val[i][j:ws+j]) 
                windows.append(val[i][j])             # a1
                windows.append(val[i][j+1])           # a2
                windows.append(val[i][-1])            # last activity/resource = outcome
                windows.append(val[i][j+ws])          # desired prediction = preds
                X.append(windows)

    new_df = pd.DataFrame(X, columns=["trace_id", "windows", "a1", "a2", "outcome", "preds"]) #, "duration"
    return new_df, np.array(y)

In [402]:
train_traces = create_traces(train_df)
short_train = drop_short_cases(train_traces, "activity")
window_df1, x_train = build_windows(short_train, 4, "activity")

window_df1.head()

,trace_id,windows,a1,a2,outcome,preds
0,173688,"[26, 27, 2, 16]",26,27,34,7
1,173688,"[27, 2, 16, 7]",27,2,34,14
2,173688,"[2, 16, 7, 14]",2,16,34,18
3,173688,"[16, 7, 14, 18]",16,7,34,32
4,173688,"[7, 14, 18, 32]",7,14,34,25


In [403]:
x_train = torch.from_numpy(x_train).float()

y_train = np.array(window_df1["preds"])
y_train = torch.from_numpy(y_train).long()

print(np.array(x_train).shape, y_train.shape)

(167517, 4) torch.Size([167517])


In [443]:
# Hyperparameters:

n, m = x_train.shape
nh = 32      # number of hidden units
bs = 64      # batchsize
lr = 0.5     # learning rate
epochs = 10  # how many epochs to train for

In [444]:
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()
loss_func = F.cross_entropy

In [445]:
class Optimizer():
    def __init__(self, params, lr=0.5): self.params,self.lr=list(params),lr
        
    def step(self):
        with torch.no_grad():
            for p in self.params: p -= p.grad * lr

    def zero_grad(self):
        for p in self.params: p.grad.data.zero_()

In [446]:
def get_model():
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,38))
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
#loss_func(model(xb), yb)

In [447]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

In [448]:
fit()

In [449]:
loss, acc = loss_func(model(x_train), y_train), accuracy(model(x_train), y_train)
loss,acc

(tensor(2.9622, grad_fn=<NllLossBackward>), tensor(0.1124))